## データ前処理

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
import random

In [2]:
dataset=pd.read_csv('sample_data.csv')
dataset.tail()

,物件名,カテゴリー,住所,最寄駅0,最寄駅1,最寄駅2,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積
4343,東京ツインパークスライトウイング,賃貸マンション,東京都港区東新橋１,ＪＲ山手線/新橋駅 歩10分,都営大江戸線/汐留駅 歩3分,ＪＲ山手線/浜松町駅 歩9分,築21年,地下2地上47階建,13階,35.5万円,20000円,71万円,35.5万円,1LDK,65.81m2
4344,東京ツインパークスライトウイング,賃貸マンション,東京都港区東新橋１,ＪＲ山手線/新橋駅 歩10分,都営大江戸線/汐留駅 歩3分,ＪＲ山手線/浜松町駅 歩9分,築21年,地下2地上47階建,21階,43万円,-,86万円,43万円,3LDK,80.51m2
4345,東京ツインパークスライトウイング,賃貸マンション,東京都港区東新橋１,ＪＲ山手線/新橋駅 歩10分,都営大江戸線/汐留駅 歩3分,ＪＲ山手線/浜松町駅 歩9分,築21年,地下2地上47階建,42階,110万円,-,330万円,110万円,3LDK,148.29m2
4346,東京ツインパークスライトウイング,賃貸マンション,東京都港区東新橋１,ＪＲ山手線/新橋駅 歩10分,都営大江戸線/汐留駅 歩3分,ＪＲ山手線/浜松町駅 歩9分,築21年,地下2地上47階建,47階,160万円,-,160万円,160万円,3LDK,198.89m2
4347,東京ツインパークスライトウイング,賃貸マンション,東京都港区東新橋１,ＪＲ山手線/新橋駅 歩10分,都営大江戸線/汐留駅 歩3分,ＪＲ山手線/浜松町駅 歩9分,築21年,地下2地上47階建,47階,180万円,-,180万円,180万円,3SLDK,198.89m2


* カテゴリーはカテゴリカル変数に変換
* 住所は23区と都市名 
* 最寄駅は最も近い駅名とその時間 
* 築年数はint型に変換 
* 構造、階数、賃料、管理費、敷金、礼金はint型に変換 
* 間取りはカテゴリカル変数に変換 
* 面積は単位なしのfloat型に変換


In [3]:
def preprocessing(df):
    df_new=df.copy()
    
    wards=[]
    citys=[]
    lines=[]
    stations=[]
    distances=[]
    results=[]
    ages=[]
    builds=[]
    Floors=[]
    prices=[]
    kanrihis=[]
    sikikins=[]
    reikins=[]
    areas=[]
    
    for i in range(0,len(df_new)):
        ###住所を'23区'、'都市名'に分ける
        pattern='.*区'
        repatter = re.compile(pattern)
        result = repatter.match(df['住所'][i])        
        wards.append(result.group()[3:])
        citys.append(df['住所'][i].split('区')[1])
    
        ###最寄駅は最も近い駅名、路線と時間を採用する
#         nearest_station=df['最寄駅0'][i]
#         lines.append(nearest_station.split('/')[0])
#         stations.append(nearest_station.split('/')[1].split(' ')[0])
#         distances.append(int(nearest_station.split(' ')[1][1]))
        
        access=dataset['最寄駅0'][i]
    
        #路線
        line=access.split('/')[0]
    #     print(line)

        #駅と徒歩
        station_distance=access.split('/')[1]
#         print(station_distance)

        #駅
        station=re.findall(r'.*駅', station_distance)
#         print(station)

        #徒歩
        distance=re.findall(r'\d+',access)
    #     print(int(distance[0]))

        lines.append(line)
        if len(station)==1:
            stations.append(station[0])
        else:
            stations.append(station_distance.split(' ')[0])
#             print('================================================================')
            
        distances.append(int(distance[0]))

        
        ###築年数はint型に変換
        age=dataset['築年数']
        if age[i][0]=='新':
            ages.append(0)
        else:   
            ages.append(int(age[i].split('年')[0][1:]))
        
        ###構造
        m=re.findall(r'\d+', dataset['構造'][i])
        if len(m)==1:
            builds.append(int(m[0]))
        elif len(m)==2:
            builds.append(int(m[1]))
        else:
            builds.append(1)

        ###部屋の階数
        floor=dataset['階数'][i]
        f=re.findall(r'\d+',floor)

        if len(f)==1:
            Floors.append(int(f[0]))
        else:
            Floors.append(random.randint(1,int(builds[i])))
        
        ###賃料
        price=dataset['賃料']
        prices.append(float(price[i].split('万')[0])*10000)
        
        ###管理費
        kanrihi=dataset['管理費']
        if kanrihi[i]!='-':
            kanrihis.append(int(kanrihi[i].split('円')[0]))
        else:
            kanrihis.append(float(0))
        
        ###敷金
        sikikin=dataset['敷金']
        if sikikin[i]!='-':
            sikikins.append(float(sikikin[i].split('万')[0])*10000)
        else:
            sikikins.append(float(0))
                
        ###礼金
        reikin=dataset['礼金']
        if reikin[i]!='-':
            reikins.append(float(reikin[i].split('万')[0])*10000)
        else:
            reikins.append(float(0))
        
        ###面積
        area=dataset['面積']
        areas.append(float(area[i].split('m')[0]))
        
        
    ###変換したカラムの追加
    df_new['23区']=wards
    df_new['都市名']=citys
    df_new['路線']=lines
    df_new['最寄駅']=stations
    df_new['徒歩']=distances
    df_new['築年数']=ages
    df_new['構造']=builds
    df_new['階数']=Floors
    df_new['賃料']=prices
    df_new['管理費']=kanrihis
    df_new['敷金']=sikikins
    df_new['礼金']=reikins
    df_new['面積']=areas
    
#     ###カテゴリカル変数に変換
#     df_new['カテゴリー']=df['カテゴリー'].astype('category') 
#     df_new['間取り']=df_new['間取り'].astype('category')
#     df_new['23区']=df_new['23区'].astype('category')
#     df_new['都市名']=df_new['都市名'].astype('category')
#     df_new['路線']=df_new['路線'].astype('category')
#     df_new['最寄駅']=df_new['最寄駅'].astype('category')

    ###不要なカラムを削除
    df_new=df_new.drop('住所',axis=1)
    df_new=df_new.drop(['最寄駅0', '最寄駅1', '最寄駅2'],axis=1)    
    
    return df_new

df_new=preprocessing(dataset)
df_new.head()

,物件名,カテゴリー,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積,23区,都市名,路線,最寄駅,徒歩
0,プチ・ボヌール,賃貸アパート,4,2,2,148000.0,5000.0,148000.0,148000.0,42DK,48.87,大田区,南蒲田３,京急空港線,糀谷駅,7
1,Ｃａｓａ荒川 ｐａｒｔ３,賃貸アパート,8,3,1,88000.0,3000.0,0.0,88000.0,1SK,31.65,荒川区,荒川１,東京メトロ千代田線,町屋駅,14
2,コンフォリア町屋,賃貸マンション,16,11,2,121000.0,7000.0,121000.0,0.0,1LDK,39.00,荒川区,荒川５,東京メトロ千代田線,町屋駅,8
3,コンフォリア町屋,賃貸マンション,16,11,5,122000.0,7000.0,122000.0,0.0,1LDK,39.00,荒川区,荒川５,東京メトロ千代田線,町屋駅,8
4,ＭＡＩＳＯＮ．Ｏ．,賃貸マンション,0,6,2,131000.0,10000.0,131000.0,131000.0,1LDK,42.32,荒川区,南千住１,東京メトロ日比谷線,三ノ輪駅,8


In [4]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4348 entries, 0 to 4347
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   物件名     4348 non-null   object 
 1   カテゴリー   4348 non-null   object 
 2   築年数     4348 non-null   int64  
 3   構造      4348 non-null   int64  
 4   階数      4348 non-null   int64  
 5   賃料      4348 non-null   float64
 6   管理費     4348 non-null   float64
 7   敷金      4348 non-null   float64
 8   礼金      4348 non-null   float64
 9   間取り     4348 non-null   object 
 10  面積      4348 non-null   float64
 11  23区     4348 non-null   object 
 12  都市名     4348 non-null   object 
 13  路線      4348 non-null   object 
 14  最寄駅     4348 non-null   object 
 15  徒歩      4348 non-null   int64  
dtypes: float64(5), int64(4), object(7)
memory usage: 543.6+ KB


In [5]:
df_new.to_csv('preprocessed_data.csv', index=None)

In [28]:
### 構造の変換
for i in range(len(dataset)):
    s=dataset['構造'][i]
    m=re.findall(r'\d+',s)
#     if len(m)==1:
#         print(int(m[0]))
#     else:
#         print('===============')
#         print('地下：', int(m[0]))
#         print('地上：', int(m[1]))
#         print('===============')

In [53]:
### 階数の変換
Floors=[]

for i in range(len(dataset)):
#     s=dataset['階数'][i]
#     t=dataset['構造'][i]
    
#     m=re.findall(r'\d+',s)
#     n=re.findall(r'\d+',t)
    
#     if len(m)==1:
#         print(int(m[0]))
#     else:
#         print('階数表記なし', random.randint(1,int(n[0])))
        
    floor=dataset['階数'][i]
    f=re.findall(r'\d+',floor)

    if len(floor)==1:
        Floors.append(int(f[0]))
    else:
#         Floors.append(random.randint(1,int(builds[i])))
        print('階数はわからない')

print(Floors)

階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階

階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階数はわからない
階

IndexError: list index out of range

In [43]:
lines=[]
stations=[]
distances=[]

#最寄駅の変換
for i in range(len(dataset)):
    access=dataset['最寄駅0'][i]
    
    #路線
    line=access.split('/')[0]
#     print(line)
    
    #駅と徒歩
    station_distance=access.split('/')[1]
#     print(station_distance)
    
    #駅
    station=re.findall(r'.*駅', station_distance)
#     print(station[0])
    
    #徒歩
    distance=re.findall(r'\d+',access)
#     print(int(distance[0]))
    
    lines.append(line)
    stations.append(station[0])
    distances.append(int(distance[0]))
    
    print(lines[i], stations[i], distances[i])

京急空港線 糀谷駅 7
東京メトロ千代田線 町屋駅 14
東京メトロ千代田線 町屋駅 8
東京メトロ千代田線 町屋駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ千代田線 町屋駅 10
東京メトロ千代田線 町屋駅 10
東京メトロ千代田線 町屋駅 10
東京メトロ千代田線 町屋駅 10
京成本線 堀切菖蒲園駅 8
ＪＲ山手線 日暮里駅 4
ＪＲ山手線 日暮里駅 4
東京メトロ日比谷線 三ノ輪駅 6
東京メトロ日比谷線 三ノ輪駅 6
東京メトロ日比谷線 三ノ輪駅 6
東京メトロ日比谷線 三ノ輪駅 6
東京メトロ日比谷線 三ノ輪駅 6
東京メトロ日比谷線 三ノ輪駅 6
東京メトロ日比谷線 三ノ輪駅 6
東京メトロ日比谷線 三ノ輪駅 6
ＪＲ常磐線 南千住駅 5
ＪＲ常磐線 南千住駅 5
東京メトロ千代田線 北綾瀬駅 6
東京メトロ千代田線 北綾瀬駅 6
東京メトロ千代田線 北綾瀬駅 6
東京メトロ千代田線 北綾瀬駅 6
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ日比谷線 三ノ輪駅 8
東京メトロ東西線 西葛西駅 8
東京メトロ東西線 西葛西駅 8
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
東京メトロ千代田線 町屋駅 5
つくばエクスプレス 六町駅 6
東急東横線 都立大学駅 3
東急東横線 都立大学駅 3
東急東横線 都立大学駅 3
東急東横線 都立大学駅 3
ＪＲ山手線 日暮里駅 9
東京メトロ日比谷線 三ノ輪駅 5
東京メ

東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ銀座線 浅草駅 7
東京メトロ日比谷線 北千住駅 8
東京メトロ日比谷線 北千住駅 8
東京メトロ日比谷線 北千住駅 8
東京メトロ日比谷線 北千住駅 8
東京メトロ日比谷線 北千住駅 8
東京メトロ日比谷線 北千住駅 8
東京メトロ日比谷線 北千住駅 8
東京メトロ日比谷線 北千住駅 8
東武東上線 大山駅 7
西武池袋線 練馬高野台駅 10
西武池袋線 練馬高野台駅 10
都営浅草線 本所吾妻橋駅 2
都営浅草線 本所吾妻橋駅 2
都営浅草線 本所吾妻橋駅 2
都営浅草線 本所吾妻橋駅 2
都営浅草線 本所吾妻橋駅 2
都営浅草線 本所吾妻橋駅 2
東武伊勢崎線 竹ノ塚駅 12
ＪＲ総武線 両国駅 3
ＪＲ総武線 両国駅 3
ＪＲ総武線 両国駅 3
ＪＲ総武線 両国駅 3
ＪＲ総武線 両国駅 3
都営三田線 板橋本町駅 7
都営三田線 板橋本町駅 7
都営三田線 板橋本町駅 7
都営三田線 板橋本町駅 7
都営三田線 板橋本町駅 7
つくばエクスプレス 浅草駅 5
つくばエクスプレス 浅草駅 5
ＪＲ埼京線 北赤羽駅 4
ＪＲ埼京線 北赤羽駅 4
東京メトロ日比谷線 中目黒駅 6
東京メトロ日比谷線 中目黒駅 6
東京メトロ日比谷線 中目黒駅 6
東京メトロ日比谷線 中目黒駅 6
東京メトロ日比谷線 中目黒駅 6
東京メトロ日比谷線 中目黒駅 6
東京メトロ日比谷線 中目黒駅 6
京成本線 お花茶屋駅 10
京成本線 お花茶屋駅 10
京成本線 お花茶屋駅 10
西武新宿線 下井草駅 10
日暮里・舎人ライナー 舎人駅 10
日暮里・舎人ライナー 舎人駅 10
日暮里・舎人ライナー 舎人駅 10
日暮里・舎人ライナー 舎人駅 10
日暮里・舎人ライナー

都営大江戸線 新御徒町駅 2
東京メトロ東西線 木場駅 3
ＪＲ横須賀線 西大井駅 5
東武東上線 下板橋駅 8
東武東上線 下板橋駅 8
東武東上線 下板橋駅 8
東武東上線 下板橋駅 8
東武伊勢崎線 曳舟駅 9
東武伊勢崎線 曳舟駅 9
東武伊勢崎線 曳舟駅 9
東武伊勢崎線 曳舟駅 9
東武伊勢崎線 曳舟駅 9
都営大江戸線 勝どき駅 4
都営大江戸線 勝どき駅 4
ＪＲ総武線 新小岩駅 9
都営大江戸線 豊島園駅 7
都営大江戸線 豊島園駅 7
都営大江戸線 豊島園駅 7
都営大江戸線 豊島園駅 7
都営大江戸線 豊島園駅 7
都営大江戸線 豊島園駅 7
都営大江戸線 豊島園駅 7
東京メトロ副都心線 池袋駅 2
ＪＲ山手線 五反田駅 9
東急大井町線 荏原町駅 8
東武伊勢崎線 東向島駅 5
京王線 芦花公園駅 5
京王線 芦花公園駅 5
東武伊勢崎線 東向島駅 5
東武伊勢崎線 東向島駅 5
西武池袋線 練馬駅 7
西武池袋線 練馬駅 7
西武池袋線 練馬駅 7
西武池袋線 練馬駅 7
西武池袋線 練馬駅 7
西武池袋線 練馬駅 7
東京メトロ丸ノ内線 中野新橋駅 5
東京メトロ丸ノ内線 方南町駅 9
東京メトロ丸ノ内線 方南町駅 9
ＪＲ山手線 目黒駅 9
ＪＲ山手線 目黒駅 9
ＪＲ山手線 目黒駅 9
ＪＲ山手線 目黒駅 9
ＪＲ山手線 目黒駅 9
ＪＲ山手線 目黒駅 9
西武新宿線 下井草駅 3
つくばエクスプレス 浅草駅 6
東京メトロ副都心線 雑司が谷駅 1
東京メトロ副都心線 雑司が谷駅 1
東京メトロ副都心線 雑司が谷駅 1
東京メトロ副都心線 雑司が谷駅 1
東京メトロ副都心線 雑司が谷駅 1
東京メトロ副都心線 雑司が谷駅 1
東京メトロ副都心線 雑司が谷駅 1
東京メトロ東西線 葛西駅 4
東京メトロ東西線 葛西駅 4
ＪＲ中央線 荻窪駅 8
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 菊川駅 3
都営新宿線 

東京メトロ有楽町線 豊洲駅 9
東京メトロ有楽町線 豊洲駅 9
東京メトロ有楽町線 豊洲駅 9
東京メトロ有楽町線 豊洲駅 9
東京メトロ有楽町線 豊洲駅 9
東京メトロ有楽町線 豊洲駅 9
東京メトロ有楽町線 豊洲駅 9
東京メトロ有楽町線 豊洲駅 9
東京メトロ有楽町線 豊洲駅 9
東京メトロ有楽町線 豊洲駅 9
東京メトロ有楽町線 豊洲駅 9
ＪＲ中央線 荻窪駅 9
都営三田線 白山駅 6
都営三田線 白山駅 6
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
京王新線 初台駅 5
東京メトロ日比谷線 入谷駅 4
東京メトロ日比谷線 入谷駅 4
東京メトロ日比谷線 入谷駅 4
東武東上線 下板橋駅 8
東武東上線 下板橋駅 8
東武東上線 下板橋駅 8
東武東上線 下板橋駅 8
東急目黒線 不動前駅 6
東急目黒線 不動前駅 6
東急目黒線 不動前駅 6
東急目黒線 不動前駅 6
東急目黒線 不動前駅 6
都営三田線 板橋本町駅 6
都営三田線 板橋本町駅 6
東京メトロ南北線 志茂駅 8
東京メトロ南北線 志茂駅 8
東京メトロ南北線 志茂駅 8
東武東上線 北池袋駅 6
東武東上線 北池袋駅 6
東武東上線 北池袋駅 6
西武池袋線 練馬高野台駅 7
東京メトロ副都心線 雑司が谷駅 4
東京メトロ副都心線 雑司が谷駅 4
京王線 桜上水駅 8
京王線 桜上水駅 8
京王線 桜上水駅 8
京王線 桜上水駅 8
京王線 桜上水駅 8
京王線 桜上水駅 8
京王線 桜上水駅 8
京王線 桜上水駅 8
京王線 桜上水駅 8
京王線 桜上水駅 8
東京メトロ丸ノ内線 大手町駅 8
東京メトロ丸ノ内線 大手町駅 8
東京メトロ丸ノ内線 大手町駅 8
東京メトロ丸ノ内線 大手町駅 8
東京メトロ丸ノ内線 大手町駅 8
東京メトロ丸ノ内線 大手町駅 8
都営三田線 巣鴨駅 6
都営三田線 巣鴨駅 6
都営三田線 巣鴨駅 6
都営三

東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
東京メトロ東西線 東陽町駅 9
ＪＲ山手線 神田駅 3
ＪＲ山手線 神田駅 3
ＪＲ山手線 神田駅 3
東武東上線 大山駅 4
東武東上線 大山駅 4
東武東上線 大山駅 4
東武東上線 大山駅 4
東武東上線 大山駅 4
東武東上線 大山駅 4
東急田園都市線 駒沢大学駅 5
東急田園都市線 駒沢大学駅 5
京王線 桜上水駅 2
京王線 桜上水駅 2
京王線 桜上水駅 2
京王線 桜上水駅 2
京王線 桜上水駅 2
京王線 桜上水駅 2
京王線 桜上水駅 2
京王井の頭線 浜田山駅 6
ＪＲ総武線 錦糸町駅 12
都営新宿線 瑞江駅 6
都営新宿線 瑞江駅 6
都営三田線 西台駅 8
都営三田線 西台駅 8
都営三田線 西台駅 8
ＪＲ山手線 目黒駅 6
ＪＲ山手線 目黒駅 6
ＪＲ山手線 目黒駅 6
ＪＲ山手線 目黒駅 6
東京メトロ副都心線 北参道駅 5
東京メトロ副都心線 北参道駅 5
ＪＲ中央線 西荻窪駅 8
ＪＲ中央線 高円寺駅 6
ＪＲ中央線 高円寺駅 6
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ山手線 田町駅 8
ＪＲ

東急目黒線 西小山駅 6
東急目黒線 西小山駅 6
ＪＲ中央線 中野駅 9
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
ＪＲ横須賀線 西大井駅 3
都営三田線 御成門駅 6
都営三田線 御成門駅 6
ＪＲ中央線 荻窪駅 4
小田急線 参宮橋駅 3
小田急線 参宮橋駅 3
小田急線 参宮橋駅 3
小田急線 参宮橋駅 3
都営大江戸線 本郷三丁目駅 6
都営大江戸線 本郷三丁目駅 6
都営大江戸線 本郷三丁目駅 6
都営大江戸線 本郷三丁目駅 6
ＪＲ総武線 亀戸駅 10
ＪＲ総武線 亀戸駅 10
ＪＲ総武線 亀戸駅 10
ＪＲ総武線 亀戸駅 10
ＪＲ総武線 亀戸駅 10
ＪＲ総武線 亀戸駅 10
ＪＲ中央線 西荻窪駅 9
京成本線 青砥駅 1
京成本線 青砥駅 1
京成本線 青砥駅 1
東武東上線 下板橋駅 1
東武東上線 下板橋駅 1
東武東上線 下板橋駅 1
都営新宿線 西大島駅 7
都営新宿線 西大島駅 7
都営新宿線 西大島駅 7
ＪＲ山手線 大塚駅 10
ＪＲ中央線 西荻窪駅 5
ＪＲ中央線 西荻窪駅 5
ＪＲ中央線 西荻窪駅 5
ＪＲ中央線 西荻窪駅 5
ＪＲ中央線 西荻窪駅 5
西武新宿線 上井草駅 9
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線 板橋区役所前駅 1
都営三田線